In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader

In [ ]:
%run models/MCD-NILM.ipynb

In [ ]:
class EnergyDataset(Dataset):
    def __init__(self, X, y, long_tern, seasonal, short_term, train=True):
        """
        Dataset for energy disaggregation
        
        Args:
            X: Total energy consumption data of shape (n_samples, sequence_length, 1)
            y: Individual appliance consumption data of shape (n_samples, sequence_length, n_appliances)
            train: Whether this is training data (for potential data augmentation)
        """
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)
        self.long_term = torch.tensor(long_tern, dtype=torch.float32)
        self.seasonal = torch.tensor(seasonal, dtype=torch.float32)
        self.short_term = torch.tensor(short_term, dtype=torch.float32)
        self.train = train
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        X_sample = self.X[idx]
        y_sample = self.y[idx]
        long_term_sample = self.long_term[idx]
        seasonal_sample = self.seasonal[idx]
        short_term_sample = self.short_term[idx]
        
        return X_sample, y_sample, long_term_sample, seasonal_sample, short_term_sample

In [ ]:
def standardize_data(df, cols, mean_std=None):
    """
    Standardize input and target columns in the DataFrame
    """
    if mean_std is None:
        mean_std = {}
    for col in cols:
        if mean_std is None:
            mean_std[col] = (df[col].mean(), df[col].std())
        df[col] = (df[col] - mean_std[col][0]) / mean_std[col][1]
        # mean_std[col] = (df[col].min(), df[col].max())
        # df[col] = (df[col] - mean_std[col][0]) / (mean_std[col][1] - mean_std[col][0])
    
    return torch.tensor(df.values), mean_std

def get_mean_std(df, cols):
    """
    Get mean and standard deviation for the specified columns in the DataFrame
    """
    mean_std = {}
    for col in cols:
        mean_std[col] = (df[col].mean(), df[col].std())
    
    return mean_std

def inverse_standardize_data(df, mean_std):
    """
    Inverse standardize the DataFrame columns
    """
    for col, (mean, std) in mean_std.items():
        df[col] = df[col] * std + mean
    
    return df

def prepare_data(df_input, df_targets, seq_len, stride):
    """
    Prepare data for training and testing
    """
    sequences = []
    targets = []
    long_term = []
    seasonal = []
    short_term = []

    for i in range(0, len(df_input) - seq_len, stride):
        seq = df_input[i:i + seq_len]
        lt = df_input[i:i + seq_len]
        se = df_input[i:i + 10]
        st = df_input[i:i + 5]

        long_term.append(lt)
        seasonal.append(se)
        short_term.append(st)
        target = df_targets[i: i + seq_len]
        sequences.append(seq)
        targets.append(target)

    long_term = np.array(long_term)
    seasonal = np.array(seasonal)
    short_term = np.array(short_term)
    residuals_input = [long_term, seasonal, short_term]
    sequences = np.array(sequences)
    targets = np.array(targets)

    return sequences, targets, residuals_input


In [ ]:
def reconstruct(y, width=30, strides=1, merge_type="mean"):
    
    len_total = width+(y.shape[0]-1)*strides
    depth = width//strides
    
    yr = np.zeros([len_total, depth])
    yr[:] = np.nan
    
    for i in range(y.shape[0]):
        for d in range(depth):
            yr[i*strides+(d*strides):i*strides+((d+1)*strides),d] = y[i,d*strides:(d+1)*strides,0]
    if merge_type == "mean":
        yr = np.nanmean(yr, axis=1)
    else:
        yr = np.nanmedian(yr, axis=1)
    
    return yr

In [ ]:
data = pd.read_csv('data/661_compressed.csv')
# data_test = pd.read_csv('../data/661_compressed.csv')
# data['weekend'] = data['local_15min'].apply(lambda x: 1 if pd.to_datetime(x).dayofweek >= 5 else 0)
# data['hour'] = data['local_15min'].apply(lambda x: pd.to_datetime(x).hour)
# data['minute'] = data['local_15min'].apply(lambda x: pd.to_datetime(x).minute)
# # # morning from 6 -12, afternoon from 12-17, evening from 17-20, night from 20-6
# # data['time_of_day'] = data['hour'].apply(lambda x: 0 if 6 <= x < 12 else (1 if 12 <= x < 17 else (2 if 17 <= x < 20 else 3)))
# data['day_of_week'] = data['local_15min'].apply(lambda x: pd.to_datetime(x).dayofweek + 1)  # Monday=1, Sunday=7

In [ ]:
# add a column for season in usa -> summar -> may - august
# data['season'] = data['local_15min'].apply(lambda x: 1 if 5 <= pd.to_datetime(x).month <= 8 else 0)
data['season'] = data['local_15min'].apply(lambda x: 0 if 3 <= pd.to_datetime(x).month < 6 else (1 if 6 <= pd.to_datetime(x).month < 9 else (2 if 9 <= pd.to_datetime(x).month < 12 else 3)))

In [ ]:
train_col = ['total_usage']
test_col = ['refrigerator', 'ev_car', 'others', 'air', 'microwave', 'dishwasher', 'clotheswasher']
# data['total_usage'] = data['total_usage'] - data['ev_car']
data[train_col + test_col] = data[train_col + test_col]
data[train_col + test_col] = data[train_col + test_col]
# data['day_of_week'] = pd.to_datetime(data['timestamp']).dt.dayofwe
# train_col = train_col + ['weekend', 'hour', 'day_of_week', 'season', 'minute']  # Add time features

In [ ]:
train_size = int(len(data) * 0.7)
test_size = len(data) - train_size
X_train = data.loc[:train_size, train_col]
y_train = data.loc[:train_size, test_col]
X_test = data.loc[train_size:, train_col]
y_test = data.loc[train_size:, test_col]

mean_std_X = get_mean_std(data, train_col)
mean_std_Y = get_mean_std(data, test_col)

# mean_std_X_test = get_mean_std(data_test, train_col)
# mean_std_Y_test = get_mean_std(data_test, test_col)

x_train_standardized, scaler_x_train = standardize_data(X_train.copy(), train_col, mean_std_X)
y_train_standardized, scaler_y_train = standardize_data(y_train.copy(), test_col, mean_std_Y)

x_test_standardized, scaler_x_test = standardize_data(X_test.copy(), train_col, mean_std_X)
y_test_standardized, scaler_y_test = standardize_data(y_test.copy(), test_col, mean_std_Y)

x_processed_train, y_processed_train, residual_train = prepare_data(
    x_train_standardized, y_train_standardized, seq_len=30, stride=1)

x_processed_test, y_processed_test, residual_test = prepare_data(
    x_test_standardized, y_test_standardized, seq_len=30, stride=1)

In [ ]:
train_data = EnergyDataset(x_processed_train, y_processed_train, residual_train[0], residual_train[1], residual_train[2], train=True)
test_data = EnergyDataset(x_processed_test, y_processed_test, residual_test[0], residual_test[1], residual_test[2], train=False)

In [ ]:
train_dataloader = DataLoader(train_data, batch_size=256, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=256, shuffle=False)

In [ ]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

def entropy_loss(weights):
    entropy = -torch.sum(weights * torch.log(weights + 1e-8), dim=-1)
    return entropy.mean()

def kl_uniform_loss(weights):
    B, T, K = weights.shape
    uniform = torch.full_like(weights, 1.0 / K)
    return F.kl_div(weights.log(), uniform, reduction='batchmean')

model = MCDNILM(input_dim=1, hidden_dim=64)
# model.apply(init_weights)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.MSELoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)


In [ ]:
import time

# Training loop
num_epochs = 5
total_time = 0.0
torch.cuda.reset_peak_memory_stats()

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    start_time = time.time()
    for inputs, targets, _, _, _ in train_dataloader:
        inputs = inputs.to(device)  # Shape: (batch_size, 1, window_size)
        targets = targets.to(device)  # Shape: (batch_size, num_appliances)
        
        optimizer.zero_grad()
        outputs, weights = model(inputs)
        loss = criterion(outputs, targets)
        loss2 = F.l1_loss(outputs, targets)
        loss3 = entropy_loss(weights)  # Calculate entropy loss
        loss = loss +  loss2 + 0.01 * loss3# Combine MSE and L
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
    end_time = time.time()
    total_time += end_time - start_time
    epoch_loss = running_loss / len(train_data)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Time: {end_time - start_time:.2f}s')
print("🔁 Peak GPU memory used during 1 epoch (training):", torch.cuda.max_memory_allocated() / (1024 ** 2), "MB")
print(f'Average time per epoch: {total_time / num_epochs:.2f}s')

In [ ]:
import psutil
import os

# testing loop
model.eval()
running_loss = 0.0
targets_list = []
outputs_list = []
process = psutil.Process(os.getpid())
mem_before = process.memory_info().rss

for inputs, targets, _, _, _ in test_dataloader:
    inputs = inputs.to(device)  # Shape: (batch_size, 1, window_size)
    targets = targets.to(device)  # Shape: (batch_size, num_appliances)
    
    with torch.no_grad():
        outputs, _ = model(inputs)
        loss = criterion(outputs, targets)
    
    targets_list.append(targets.cpu().numpy())
    outputs_list.append(outputs.cpu().numpy())
    running_loss += loss.item() * inputs.size(0)

mem_after = process.memory_info().rss
print("🧠 PyTorch inference memory (CPU):", (mem_after - mem_before) / (1024 ** 2), "MB")

In [ ]:
targets_array = np.concatenate(targets_list, axis=0)
outputs_array = np.concatenate(outputs_list, axis=0)

In [ ]:
targets_array_reverse = targets_array.copy()
outputs_array_reverse = outputs_array.copy()

for index, key in enumerate(scaler_y_test):
    # Reverse scale in place for each feature dimension
    targets_array_reverse[:, :, index] = targets_array[:, :, index] * scaler_y_test[key][1] + scaler_y_test[key][0]
    outputs_array_reverse[:, :, index] = outputs_array[:, :, index] * scaler_y_test[key][1] + scaler_y_test[key][0]


In [ ]:
targets_array_reconstruct = []
outputs_array_reconstruct = []

for i in range(targets_array_reverse.shape[-1]):
    targets_array_reconstruct.append(reconstruct(targets_array_reverse[:, :, i].reshape(*targets_array_reverse[:, :, 0].shape, 1), width=30, strides=1, merge_type="mean"))
    outputs_array_reconstruct.append(reconstruct(outputs_array_reverse[:, :, i].reshape(*targets_array_reverse[:, :, 0].shape, 1), width=30, strides=1, merge_type="mean"))
targets_array_reconstruct = np.array(targets_array_reconstruct).T
outputs_array_reconstruct = np.array(outputs_array_reconstruct).T


In [ ]:
inverse_targets_tensor = torch.tensor(targets_array_reconstruct, dtype=torch.float32)
inverse_outputs_tensor = torch.tensor(outputs_array_reconstruct, dtype=torch.float32)

In [ ]:
# zero all negative values in the inverse outputs tensor
inverse_outputs_tensor[inverse_outputs_tensor < 0] = 0.0
# zero all negative values in the inverse targets tensor
inverse_targets_tensor[inverse_targets_tensor < 0] = 0.0

In [ ]:
import matplotlib.pyplot as plt
# Plotting the results for the first appliance (e.g., refrigerator)
plt.figure(figsize=(12, 6))
plt.plot(inverse_targets_tensor[:500000, 0], label='Actual Refrigerator Consumption', color='blue')
plt.plot(inverse_outputs_tensor[:500000, 0], label='Predicted Refrigerator Consumption', color='orange')
plt.title('Refrigerator Consumption Prediction')
plt.xlabel('Time Steps')
plt.ylabel('Power Consumption (kW)')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import r2_score
for i, val in enumerate(test_col):
    ground_truth = inverse_targets_tensor[:, i]
    predicted = inverse_outputs_tensor[:, i]
    mae_loss_value = np.abs(ground_truth - predicted).mean()
    # mae_loss_value = mae_loss(ground_truth, predicted)

    num = np.abs(ground_truth - predicted).sum(axis=0)
    den = 2*ground_truth.sum(axis=0)
    eac = (1 - num/den)

    nde = torch.sum((ground_truth - predicted) ** 2) / torch.sum((ground_truth ** 2), axis=0)
    print(f'Appliance {val} - MAE Loss: {mae_loss_value:.4f}, EAC: {eac:.4f}, NDE: {nde:.4f}')

In [ ]:
dataframe_test = pd.DataFrame(inverse_outputs_tensor.numpy(), columns=test_col)
dataframe_ground_truth = pd.DataFrame(inverse_targets_tensor.numpy(), columns=test_col)

In [ ]:
dataframe_test.to_csv('data/8156_ev_predict_our.csv', index=False)
dataframe_ground_truth.to_csv('data/8156_ev_ground_truth_our.csv', index=False)